In [2]:
%run init.ipynb

matchzoo version 0.0.1
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [3]:
preprocessor = mz.preprocessors.BasicPreprocessor(
    truncated_length_left=10,
    truncated_length_right=100,
    remove_stop_words=False,
    filter_low_freq=0
)

In [4]:
len(train_pack_raw)

20360

In [5]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 7469.25it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4014.38it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 2422119.95it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 8072.62it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 3848.09it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 712224.61it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 8058.47it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<00

In [6]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f77463993c8>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f77746ae278>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [7]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankCrossEntropyLoss(num_neg=10))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [8]:
padding_callback = mz.models.MatchLSTM.get_default_padding_callback()

In [9]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=5,
    num_neg=10
)
trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    sort=False,
    shuffle=True,
    callback=padding_callback
)

testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    sort=False,
    shuffle=False,
    callback=padding_callback
)

In [10]:
model = mz.models.MatchLSTM()

model.params['task'] = ranking_task
model.params['mask_value'] = 0
#model.params['embedding'] = embedding_matrix
model.params['embedding_output_dim'] = 300
model.params['embedding_input_dim'] = preprocessor.context['embedding_input_dim']

model.build()

print(model, sum(p.numel() for p in model.parameters() if p.requires_grad))

MatchLSTM(
  (embedding): Embedding(30058, 300)
  (dropout): Dropout(p=0.2)
  (input_proj): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(300, 100, bidirectional=True)
    )
  )
  (match_module): MatchModule(
    (v2_proj): Linear(in_features=200, out_features=200, bias=True)
    (proj): Linear(in_features=800, out_features=400, bias=True)
    (dropout): Dropout(p=0.2)
  )
  (mlstm_module): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(400, 100, bidirectional=True)
    )
  )
  (classification): Sequential(
    (0): Dropout(p=0.2)
    (1): Linear(in_features=200, out_features=200, bias=True)
    (2): Tanh()
  )
  (out): Linear(in_features=200, out_features=1, bias=True)
) 10141601


In [11]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [12]:
trainer.run()

[Iter-255 Loss-1.746]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5822 - normalized_discounted_cumulative_gain@5(0.0): 0.6344 - mean_average_precision(0.0): 0.5914



[Iter-510 Loss-0.644]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6041 - normalized_discounted_cumulative_gain@5(0.0): 0.6505 - mean_average_precision(0.0): 0.6139



[Iter-765 Loss-0.199]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5933 - normalized_discounted_cumulative_gain@5(0.0): 0.6365 - mean_average_precision(0.0): 0.601



[Iter-1020 Loss-0.133]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.598 - normalized_discounted_cumulative_gain@5(0.0): 0.6449 - mean_average_precision(0.0): 0.6048



[Iter-1275 Loss-0.093]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5777 - normalized_discounted_cumulative_gain@5(0.0): 0.6222 - mean_average_precision(0.0): 0.5883



[Iter-1530 Loss-0.066]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.574 - normalized_discounted_cumulative_gain@5(0.0): 0.6311 - mean_average_precision(0.0): 0.5881



[Iter-1785 Loss-0.054]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.584 - normalized_discounted_cumulative_gain@5(0.0): 0.6366 - mean_average_precision(0.0): 0.5957



[Iter-2040 Loss-0.055]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.571 - normalized_discounted_cumulative_gain@5(0.0): 0.6266 - mean_average_precision(0.0): 0.5852



[Iter-2295 Loss-0.048]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5983 - normalized_discounted_cumulative_gain@5(0.0): 0.6514 - mean_average_precision(0.0): 0.6154



[Iter-2550 Loss-0.040]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5732 - normalized_discounted_cumulative_gain@5(0.0): 0.6256 - mean_average_precision(0.0): 0.5843

Cost time: 2253.530198097229s
